# Descripción

Este notebook contiene los pasos requeridos para asignar las combinaciones, previamente asignadas, profesor-curso a un salón de acuerdo a los requerimientos institucionales.

In [1]:
import json

# READ DATA

## first stage model output

In [2]:
# basic parameters
DAY_LENGTH=16

# Horizonte de planificación
HORIZON=5

#  Días de la semana
DAYS=['Lunes','Martes','Miercoles','Jueves','Viernes']


In [3]:
# Este diccionario contiene un mapa de los slots de tiempo a los días y horas

actual_time={}
for d in range(HORIZON):
    for h in range(DAY_LENGTH):
        actual_time[d*16+h]={"slot":DAYS[d]+' '+str(h+6)+':00', "day":d, "hour":h,'slot_abr':DAYS[d][0]+str(h)}

In [4]:
# read model_output.json
allocations=json.load(open('data/model_output.json','r'))
#allocations

In [5]:
groups=allocations['courses']
teachers=allocations['teachers']
pairs=allocations['ensembles']
teachers=set([int(t) for t in teachers])
groups=set([g for g in groups])
pairs=[(int(p[0]) ,p[1]) for p in pairs]

In [6]:
pairs_dict={j:i for i,j in pairs}


In [7]:
courses_by_teacher={}
for p in pairs:
    if p[0] not in courses_by_teacher:
        courses_by_teacher[p[0]]=[p[1]]
    else:
        courses_by_teacher[p[0]].append(p[1])

# teachers by course
teachers_by_course={}
for p in pairs:
   teachers_by_course[p[1]]=p[0]




## Initial data load

In [8]:
# read data.json
horas_curso=json.load(open('data/horas_curso.json','r'))
grupo_requerimiento=json.load(open('data/grupo_requerimiento.json','r'))
ubicacion_semestral=json.load(open('data/ubicacion_semestral.json','r'))
disponibilidad_docentes=json.load(open('data/disponibilidad.json','r'))

In [9]:
tipo_salon={'A','B','C','D','E','F','G'}
hora_disponibilidad_x_salon={
    'A':10
    ,'B':0
    ,'C':0
    ,'D':10
    ,'E':10
    ,'F':10
    ,'G':10
}

cantidad_de_salones={
 'A':11
 ,'B':9
 ,'C':3
 ,'D':1
 ,'E':1
 ,'F':2
 ,'G':1   
}



salas_de_inform={"C","D"}

clases_sala_infor={
    "2027-0059",
    "9937-0012",
    "6612-0001",
    "8830-0008",
    "8837-0021",
    "8830-0006",
    "9937-0013",
    "6611-0088",
    "8827-0008",
    "8830-0009",
    "6069-0018",
    "6069-0127",
    "9937-0014",
    "8821-0010",
    "9937-0018"
    
}



In [10]:
names={326151: 'Alvaro Jose Torres Penagos',
336392:	'Andres Felipe Hernandez Giraldo',
183645:	'Andres Felipe Parra Perea',
274691:	'Beatriz Elena Hernandez Arias',
259554:	'Claudia Orly Escudero Jimenez',
331121:	'Edwin Fernando Restrepo Salazar',
219830:	'Edwin Javier Ortega Zuñiga',
187061:	'Jairo Arboleda Zuñiga',
277461:	'Mariela Galindo',
226130:	'Pedro Nel Barbosa Garcia',
226130:	'Ruben Dario Parra Zuleta',
182876:	'William Andres Alzate Cobo'}

# Modelo

In [11]:
I=set(teachers)
J=set(groups)
K=set(pairs)
C= tipo_salon

In [12]:
from itertools import product

# courses that are the same
same_course={}
# courses that are taught by the same teacher
same_teacher={}

# delta semestres
delta_semestre={}
for j1,j2 in product(J,J):
    same_course[(j1,j2)]=0
    same_teacher[(j1,j2)]=0
    if j1[:9]==j2[:9]:
        same_course[(j1,j2)]=1
    delta_semestre[(j1,j2)]=9-abs(ubicacion_semestral[j1[:9]]["Semestre"]-ubicacion_semestral[j2[:9]]["Semestre"])

sets_of_courses=[]
for p in pairs:
    for p2 in pairs:
        if p[0]==p2[0]:
            same_teacher[(p[1],p2[1])]=1
            sets_of_courses.append(set([p[1],p2[1]]))
# merge sets of courses when they have a common course
for i in range(len(sets_of_courses)):
    for j in range(i+1,len(sets_of_courses)):
        if len(sets_of_courses[i].intersection(sets_of_courses[j]))>0:
            sets_of_courses[i]=sets_of_courses[i].union(sets_of_courses[j])
            sets_of_courses[j]=set()
# remove empty sets
sets_of_courses=[s for s in sets_of_courses if len(s)>0]


# course and time slot compatibility
time_compatibility={}
for c,j in product(C,J):
    for s in actual_time.keys():
        time_compatibility[(c,j,s)]=0
        if actual_time[s]['hour'] + horas_curso[j[:9]]['horas']-1<DAY_LENGTH:
            time_compatibility[(c,j,s)]=1

print(disponibilidad_docentes)
valid_keys=[]
for c,j,s in product(C,J,actual_time.keys()):
    if time_compatibility[(c,j,s)]==1 and s in disponibilidad_docentes[str(teachers_by_course[j])]:
        valid_keys.append((c,j,s))



{'326151': [3, 4, 5, 6, 8, 9, 10, 11, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 36, 37, 38, 39, 40, 43, 44, 60, 61, 62, 63, 64, 65, 66, 76, 77, 78, 79, 80, 81, 82, 83], '336392': [3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 25, 26, 27, 28, 29, 30, 31, 32, 38, 39, 43, 44, 45, 46, 47, 48, 49, 54, 55, 56, 59, 60, 61, 62, 63, 79, 80, 81, 82, 83], '183645': [2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, 27, 28, 29, 30, 31, 32, 43, 44, 45, 46, 47, 48, 49, 55, 56, 59, 60, 61, 62, 77, 78, 79, 80, 81, 82, 87, 88, 89, 90], '274691': [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 22, 23, 28, 29, 30, 31, 32, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83], '259554': [7, 8, 9, 10, 11, 12, 13, 14, 15, 22, 23, 29, 30, 31, 32, 42, 43, 44, 45, 46, 47, 48, 49, 61, 62, 63, 64, 65, 66, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92], '331121': [7, 8, 9, 10, 11, 12, 13, 14, 15, 29, 30, 31, 32, 42, 43, 44, 45, 46, 59, 60, 61, 62, 63, 64, 75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92]

In [13]:
import gurobipy as gp

# declare an empty model
m=gp.Model('Asignacion de salones')
m.setParam('OutputFlag',False)



# declares variables

# y representa si el cursu j se asigna a un salón tipo c iniciando en el slot s
y=m.addVars(valid_keys,vtype=gp.GRB.BINARY,name='y')
# x representa si el curso j ocupa un salón tipo c durante el slot s
x=m.addVars([(c,j,s) for c in C for j in J for s in actual_time.keys()],vtype=gp.GRB.BINARY,name='x')
# indica si dos cursos se solapan en algún momento
z=m.addVars([(j1,j2) for j1,j2 in product(J,J) if j1!=j2],vtype=gp.GRB.BINARY,name='z')
# classroooms
req_rooms=m.addVars(C,vtype=gp.GRB.INTEGER,name='req_rooms')

Set parameter Username


Academic license - for non-commercial use only - expires 2024-04-04


Restricción 1: Todos los cursos deben tener exactamente una asignación de inicio.

$\sum_{c,j1,s \in \text{{valid\_keys}}, j1=j} y[(c,j1,s)] = 1 \quad \forall j \in J$


In [14]:
# each course starts at one time slot
m.addConstrs((gp.quicksum(y[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==1 for j in J),name='each_course_starts_at_one_time_slot')
0

0

Restricción 2:

si un curso da inicio durante el slot s en el salón tipo c, y tiene una duración t, ese curso debe estar ocupando un slot entre s, s+1 hasta s+t-1

$  
\sum_{s1 \in \text{{range}}(s,s+\text{{horas\_curso}}[j1[:9]]['horas']) \atop (c,j1,s1) \in \text{{valid\_keys}}} x[(c,j1,s1)] \geq y[(c,j1,s)] \cdot \text{{horas\_curso}}[j1[:9]]['horas'] \forall c, j1, s \in \text{{valid\_keys}}
$

In [15]:
"""cont=0
for c,j1,s in valid_keys:
    lhs=gp.quicksum(x[(c,j1,s1)] for s1 in range(s,s+horas_curso[j1[:9]]['horas']) )
    rhs=y[(c,j1,s)]*horas_curso[j1[:9]]['horas']
    print(lhs,">=",rhs)
    cont+=1
    if cont==16:
        break"""
    

m.addConstrs((gp.quicksum(x[(c,j1,s1)] for s1 in range(s,s+horas_curso[j1[:9]]['horas'])  )
             >=y[(c,j1,s)]*horas_curso[j1[:9]]['horas'] for c,j1,s in valid_keys))
0

0

Restricción 3:
Cada curso es asignado un bloque de tiempo equivalente a las horas que requiere


$ \sum_{c,j1,s \in \text{{valid\_keys}} \atop j1=j} x[(c,j1,s)] = \text{{horas\_curso}}[j[:9]]['horas'] \forall j \in J$



In [16]:
m.addConstrs((gp.quicksum(x[(c,j1,s)] for c,j1,s in valid_keys if j1==j)==horas_curso[j[:9]]['horas'] for j in J),name='each_course_is_assigned_the_time_it_requires')
0

0

$\forall j1, j2 \in J \quad \text{{s.t.}} \quad j1 \neq j2, \forall c1, c2 \in C, \forall s \in \text{{actual\_time.keys()}}, z[j1,j2] \geq x[c1,j1,s] + x[c2,j2,s] - 1$


In [17]:
m.addConstrs((z[j1,j2]>=x[c1,j1,s]+x[c2,j2,s]-1 for j1,j2 in product(J,J) if j1!=j2 for c1 ,c2 in product(C,C) for s in actual_time.keys()),name='si_dos_cursos_se_imparten_a_la_misma_hora_en_cualquier_salon_se_debe_activar_la_variable_z_que_lleva_cuentas_de_los_solapamientos')
0

0

In [18]:
m.addConstrs((gp.quicksum(x[(c,j,s)] for j in J )<=req_rooms[c]  for c in C for s in actual_time.keys()),name='required_rooms_of_each_type')

m.addConstrs((req_rooms[c] <=cantidad_de_salones[c]  for c in C for s in actual_time.keys()),name='cada_curso_se_imparte_en_un_maximo_de_un_salon_de_cada_tipo')
0


0

In [19]:
# si el salón requiere sala de sistemas, se debe programar en una de las salas de sistemas {"C","D"}

m.addConstrs((gp.quicksum(y[(c,j,s)] for c in salas_de_inform for s in actual_time.keys() if (c,j,s) in valid_keys)==1 for j in J if j[:9] in clases_sala_infor ),name='si_el_curso_requiere_sala_de_sistemas_se_debe_programar_en_una_de_las_salas_de_sistemas')
m.addConstrs((gp.quicksum(y[(c,j,s)] for c in salas_de_inform for s in actual_time.keys() if (c,j,s) in valid_keys)==0 for j in J if not( j[:9] in clases_sala_infor) ),name='si_el_curso_requiere_sala_de_sistemas_se_debe_programar_en_una_de_las_salas_de_sistemas')
0

0

In [20]:
# una clase no puede empezar en un slot si no está aún disponible el salón
m.addConstrs((y[(c,j,s)]==0 for c,j,s in valid_keys if actual_time[s]['hour']<hora_disponibilidad_x_salon[c]),name='una_clase_no_puede_empezar_en_un_slot_si_no_esta_aun_disponible_el_salon')
0

0

In [21]:
# two courses belonging to the same set cannot be scheduled at the same time
m.addConstrs((gp.quicksum(x[(c,j1,s)] for j1 in sets_of_courses[i] if (c,j1,s) in valid_keys)<=1 for i in range(len(sets_of_courses)) for c in C for s in actual_time.keys()), name='c1') 


{(0, 'B', 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 'B', 17): <gurobi.Constr *Awaiting Model Update*>,
 (

In [22]:
# two classes with the same code cannot take place simultaneously
m.addConstrs((y[(c1,j1,s1)]+y[(c2,j2,s2)]<=1 for (c1,j1,s1) in valid_keys for (c2,j2,s2) in valid_keys if c1==c2 and s1==s2 and j1[:9]==j2[:9] and j1!=j2), name='one_class')
m.addConstrs((x[(c1,j1,s1)]+x[(c2,j2,s2)]<=1 for (c1,j1,s1) in valid_keys for (c2,j2,s2) in valid_keys if c1==c2 and s1==s2 and j1[:9]==j2[:9] and j1!=j2), name='one_class_2')
0

0

In [23]:
m.addConstrs((z[j1,j2]==0 for j1,j2 in product(J,J) if ubicacion_semestral[j1[:9]]["Semestre"]==ubicacion_semestral[j2[:9]]["Semestre"] and j1!=j2), name='delta_semestre')
0

0

El objetivo es generar una programación tal que los cursos de semestres similares se
programan en diferentes momentos del día. Es decir, queremos maximizar la diferencia de semestre entre los cursos que se deban programar a la misma hora.

$ \max:\sum_{{j1,j2 \in J, j1 \neq j2}} z[j1,j2] \cdot \delta_{\text{{semestre}}}[j1,j2]$


In [24]:
# funcion objetivo:
# maximizar la suma de los deltas semestrales de los cursos que se imparten
obj_1=gp.quicksum(z[j1,j2]*delta_semestre[j1,j2] for j1,j2 in product(J,J) if j1!=j2)
obj_2=gp.quicksum(req_rooms[c]*(16-hora_disponibilidad_x_salon[c]) for c in C)
# model sense
m.modelSense=gp.GRB.MINIMIZE
m.setObjective(obj_2)
m.optimize()

In [ ]:

m.addConstrs((req_rooms[c] <=int(req_rooms[c].x)  for c in C for s in actual_time.keys()),name='max number of rooms')
m.modelSense=gp.GRB.MINIMIZE
m.setObjective(obj_1)


In [ ]:
time_limit=60
m.setParam('TimeLimit',time_limit)
m.optimize()

# print model status
if m.status==gp.GRB.OPTIMAL:
    print('Optimal solution found with objective: %g' % m.objVal)
elif m.status==gp.GRB.TIME_LIMIT:
    print('Time limit reached with objective: %g' % m.objVal)
else:
    print('Model is infeasible')

AttributeError: 'gurobipy.Model' object has no attribute 'getObjVal'

In [ ]:
print('Time elapsed: %g' % m.Runtime)
# print the keys for the y variables that are equal to 1
for c,j,s in valid_keys:
    if y[c,j,s].x>=0.9:
        print("la clase",horas_curso[j[:9]]['ASIGNATURA'], " grupo ",j[10:] ," se imparte en un salón tipo ",c,"en el horario del ",actual_time[s]["slot"], " con una duración de ",horas_curso[j[:9]]['horas'], " horas")

Time elapsed: 1.09332
la clase Física_Mecánica  grupo  0  se imparte en un salón tipo  G en el horario del  Viernes 18:00  con una duración de  4  horas
la clase Contexto_Profesional  grupo  0  se imparte en un salón tipo  G en el horario del  Viernes 16:00  con una duración de  2  horas
la clase OCBD_Matematica_Operativa  grupo  0  se imparte en un salón tipo  G en el horario del  Martes 16:00  con una duración de  3  horas
la clase Calculo_Diferencial  grupo  0  se imparte en un salón tipo  G en el horario del  Miercoles 18:00  con una duración de  4  horas
la clase Metodología_Investigación  grupo  0  se imparte en un salón tipo  G en el horario del  Miercoles 16:00  con una duración de  2  horas
la clase Electrotecnia_Industrial            grupo  0  se imparte en un salón tipo  G en el horario del  Jueves 16:00  con una duración de  3  horas
la clase Electiva-I_RSE  grupo  0  se imparte en un salón tipo  G en el horario del  Lunes 16:00  con una duración de  3  horas
la clase Merca

In [ ]:
required_classrooms_by_tipe = {i:0 for i in tipo_salon}
schedule_grid = {}

for c,j,s in valid_keys:
    if y[c,j,s].x>=0.9:
        # check if the classroom is already in the schedule
        if required_classrooms_by_tipe[c]==0:
            required_classrooms_by_tipe[c]+=1
            # initialize the schedule grid for the classroom with all the days and hours in null
            schedule_grid[str(c)+"-"+str(1)] = {(actual_time[s2]["slot"].split(" ")[0],actual_time[s2]["slot"].split(" ")[1]): None for s2 in actual_time.keys()}
            for s2 in range(horas_curso[j[:9]]['horas']):
                schedule_grid[str(c)+"-"+str(1)][(actual_time[s+s2]["slot"].split(" ")[0],actual_time[s+s2]["slot"].split(" ")[1])] =str(ubicacion_semestral[j[:9]]["Semestre"])+"-" +horas_curso[j[:9]]['ASIGNATURA']+"-"+(names[pairs_dict[j]] if pairs_dict[j] in names.keys() else str(pairs_dict[j]))
                # split string on space

        else:
            # checks wheter the classrooms is available for the course hours, if so it adds the course to the schedule otherwise it adds a new classroom
            for i in range(1,required_classrooms_by_tipe[c]+1):
                fits = True
                for s2 in range(horas_curso[j[:9]]['horas']):
                    if schedule_grid[str(c)+"-"+str(i)][(actual_time[s+s2]["slot"].split(" ")[0],actual_time[s+s2]["slot"].split(" ")[1])] != None:
                        fits = False
                        break
                if fits:
                    for s2 in range(horas_curso[j[:9]]['horas']):
                        schedule_grid[str(c)+"-"+str(i)][(actual_time[s+s2]["slot"].split(" ")[0],actual_time[s+s2]["slot"].split(" ")[1])] = str(ubicacion_semestral[j[:9]]["Semestre"])+"-"+horas_curso[j[:9]]['ASIGNATURA']+"-"+(names[pairs_dict[j]] if pairs_dict[j] in names.keys() else str(pairs_dict[j]))
                    break
                elif i == required_classrooms_by_tipe[c]:
                    required_classrooms_by_tipe[c]+=1
                    schedule_grid[str(c)+"-"+str(required_classrooms_by_tipe[c])] = {(actual_time[s2]["slot"].split(" ")[0],actual_time[s2]["slot"].split(" ")[1]): None for s2 in actual_time.keys()}
                    for s2 in range(horas_curso[j[:9]]['horas']):
                        schedule_grid[str(c)+"-"+str(i+1)][(actual_time[s+s2]["slot"].split(" ")[0],actual_time[s+s2]["slot"].split(" ")[1])] = str(ubicacion_semestral[j[:9]]["Semestre"])+"-"+horas_curso[j[:9]]['ASIGNATURA']+"-"+(names[pairs_dict[j]] if pairs_dict[j] in names.keys() else str(pairs_dict[j]))
                    break
required_classrooms_by_tipe                    

{'G': 1, 'C': 1, 'D': 0, 'E': 0, 'F': 1, 'A': 0, 'B': 1}

In [ ]:
import pandas as pd
list_dfs=[]
# present the actual time data as an square matrix
for sal in required_classrooms_by_tipe.keys():
    for slot in range(1,required_classrooms_by_tipe[sal]+1):
        sal_name=str(sal)+'-'+str(slot)
        

        df=[]

        for h in range(DAY_LENGTH):
            df_line=[sal_name]
            df_line.append(str(h+6)+':00')
            for d in DAYS:
                #print(schedule_grid[sal_name].keys())
                if sal_name in schedule_grid.keys() and (d,str(h+6)+':00') in schedule_grid[sal_name].keys() and schedule_grid[sal_name][(d,str(h+6)+':00')] is not None:
                    df_line.append(schedule_grid[sal_name][(d,str(h+6)+':00')].replace('_',' '))
                else:
                    df_line.append("-")
            
            df.append(df_line)
        df=pd.DataFrame(df,columns=["classroom",'time']+DAYS)
        # save to an excel file app
        list_dfs.append(df)
        display(df)

df=pd.concat(list_dfs)
df.to_excel("schedule.xlsx",index=False)

,classroom,time,Lunes,Martes,Miercoles,Jueves,Viernes
0,G-1,6:00,-,-,-,-,-
1,G-1,7:00,-,-,-,-,-
2,G-1,8:00,-,-,-,-,-
3,G-1,9:00,-,-,-,-,-
4,G-1,10:00,-,-,-,-,-
5,G-1,11:00,-,-,-,-,-
6,G-1,12:00,-,-,-,-,-
7,G-1,13:00,-,-,-,-,-
8,G-1,14:00,-,-,-,-,-
9,G-1,15:00,-,-,-,-,-


,classroom,time,Lunes,Martes,Miercoles,Jueves,Viernes
0,C-1,6:00,-,-,1-Fundamentos dibujo CAD-234541,8-Investigación Operativa-437130,-
1,C-1,7:00,-,-,1-Fundamentos dibujo CAD-234541,8-Investigación Operativa-437130,-
2,C-1,8:00,5-Modelos Regresón Series Tiempo-Andres Felipe...,-,-,8-Investigación Operativa-437130,5-Costos Presupuestos-Claudia Orly Escudero Ji...
3,C-1,9:00,5-Modelos Regresón Series Tiempo-Andres Felipe...,-,-,8-Investigación Operativa-437130,5-Costos Presupuestos-Claudia Orly Escudero Ji...
4,C-1,10:00,5-Modelos Regresón Series Tiempo-Andres Felipe...,-,-,-,5-Costos Presupuestos-Claudia Orly Escudero Ji...
5,C-1,11:00,5-Modelos Regresón Series Tiempo-Andres Felipe...,-,-,-,5-Costos Presupuestos-Claudia Orly Escudero Ji...
6,C-1,12:00,-,6-Planeación Control Producción-Alvaro Jose To...,-,7-Optimización-Jairo Arboleda Zuñiga,1-Fundamentos dibujo CAD-234541
7,C-1,13:00,6-Diseño Experimental-Edwin Fernando Restrepo ...,6-Planeación Control Producción-Alvaro Jose To...,8-Gerencia Proyectos-Edwin Javier Ortega Zuñiga,7-Optimización-Jairo Arboleda Zuñiga,1-Fundamentos dibujo CAD-234541
8,C-1,14:00,6-Diseño Experimental-Edwin Fernando Restrepo ...,6-Planeación Control Producción-Alvaro Jose To...,8-Gerencia Proyectos-Edwin Javier Ortega Zuñiga,7-Optimización-Jairo Arboleda Zuñiga,-
9,C-1,15:00,6-Diseño Experimental-Edwin Fernando Restrepo ...,6-Planeación Control Producción-Alvaro Jose To...,8-Gerencia Proyectos-Edwin Javier Ortega Zuñiga,7-Optimización-Jairo Arboleda Zuñiga,-


,classroom,time,Lunes,Martes,Miercoles,Jueves,Viernes
0,F-1,6:00,-,-,-,-,-
1,F-1,7:00,-,-,-,-,-
2,F-1,8:00,-,-,-,-,-
3,F-1,9:00,-,-,-,-,-
4,F-1,10:00,-,-,-,-,-
5,F-1,11:00,-,-,-,-,-
6,F-1,12:00,-,-,-,-,-
7,F-1,13:00,-,-,-,-,-
8,F-1,14:00,-,-,-,-,-
9,F-1,15:00,-,-,-,-,-


,classroom,time,Lunes,Martes,Miercoles,Jueves,Viernes
0,B-1,6:00,-,-,2-Economía-284431,-,8-Ética Profesional-Claudia Orly Escudero Jimenez
1,B-1,7:00,-,-,2-Economía-284431,4-Probabilidad Inferencia Estadistica-Andres F...,8-Ética Profesional-Claudia Orly Escudero Jimenez
2,B-1,8:00,4-Ecuaciones Diferenciales-Andres Felipe Parra...,-,-,4-Probabilidad Inferencia Estadistica-Andres F...,2-Fundamentos Quimica-404903
3,B-1,9:00,4-Ecuaciones Diferenciales-Andres Felipe Parra...,4-Procesos Industriales-Alvaro Jose Torres Pen...,-,4-Probabilidad Inferencia Estadistica-Andres F...,2-Fundamentos Quimica-404903
4,B-1,10:00,4-Ecuaciones Diferenciales-Andres Felipe Parra...,4-Procesos Industriales-Alvaro Jose Torres Pen...,5-ElectivaCFH-I Catedra sostenibilidad-Edwin J...,4-Probabilidad Inferencia Estadistica-Andres F...,2-Fundamentos Quimica-404903
5,B-1,11:00,4-Ecuaciones Diferenciales-Andres Felipe Parra...,4-Procesos Industriales-Alvaro Jose Torres Pen...,5-ElectivaCFH-I Catedra sostenibilidad-Edwin J...,3-Electricidad Magnetismo-Edwin Javier Ortega ...,2-Fundamentos Quimica-404903
6,B-1,12:00,9-OCI-III Htas Toma Desiciones-Beatriz Elena H...,4-Procesos Industriales-Alvaro Jose Torres Pen...,2-Economía-284431,3-Electricidad Magnetismo-Edwin Javier Ortega ...,3-Legislación-William Andres Alzate Cobo
7,B-1,13:00,9-OCI-III Htas Toma Desiciones-Beatriz Elena H...,10-OCP-II Audit.Admon Diagnos Organizacional-C...,2-Economía-284431,3-Electricidad Magnetismo-Edwin Javier Ortega ...,3-Legislación-William Andres Alzate Cobo
8,B-1,14:00,9-OCI-III Htas Toma Desiciones-Beatriz Elena H...,10-OCP-II Audit.Admon Diagnos Organizacional-C...,7-Diseño Mantenimiento Plantas -Alvaro Jose T...,1-Geometría Analítica-Andres Felipe Hernandez ...,1-Introducción Ingeniería Industrial-William A...
9,B-1,15:00,-,10-OCP-II Audit.Admon Diagnos Organizacional-C...,7-Diseño Mantenimiento Plantas -Alvaro Jose T...,1-Geometría Analítica-Andres Felipe Hernandez ...,1-Introducción Ingeniería Industrial-William A...
